# Spectral Clustering

<div class="alert alert-info" role="alert">
    <h2>Muestra muy poco balanceada</h2>
    Cogiendo una fracción de los datos, los casos de fraude seran muy reducidos 
</div>

In [3]:
a = float(input())
print(type(a))

0.1
<class 'float'>


In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.decomposition import PCA
from sklearn.cluster import SpectralClustering
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
def balancedCut(df ,n ,frac = [0.5], target = 'Class',classes=[0,1]):
    print(f'{"="*50}\nbalancedCut\n{"="*50}')
    print(f'Realizando seleccion de la muestra la muestra')
    newDfs = []
    if sum(frac) > 1:
        print(f'Valor de la fraccion incorrecto: --> {sum(frac)} > 1')
        
    if len(frac) < len(classes):
        frac.append(1-sum(frac))
    
    for cl,fc in zip(classes, frac):
        ndf = df[df[target]==cl]
        ro = ndf.shape[0]
        print(f'Dataset de clase --> {cl}')
        if int(fc*n) > ro:
            newDfs.append(ndf)
            print(f'\tFilas: {ro}')
        else:
            newDfs.append(ndf.sample(n=int(fc*n)))
            print(f'\tFilas: {int(fc*n)}')
    
    return pd.concat(newDfs)

In [ ]:
cards = pd.read_csv("/home/vcalzada/Work/proyecto/modelos/creditcard.csv")
cards = cards.sample(frac=1)


cards.head()

df = balancedCut(cards, 1000, frac=[0.9])

In [ ]:
fig = px.scatter_3d(df, x='V1',y='V2',z='V3', color='Class')
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()

In [ ]:
campos = list(cards.keys())
campos.remove("Class")
X = np.array(df[campos])
y = np.array(df["Class"])
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
y_pred = SpectralClustering(n_clusters=2,assign_labels='discretize',random_state=0).fit_predict(X[:,1:10])

In [ ]:
cm = confusion_matrix(y, y_pred)

disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=[])
disp.plot()

In [ ]:
fig = make_subplots(rows=1, cols=2,
                   specs=[[{'type': 'scene'}, {'type': 'scene'}]])
t = y_pred==0
n = y_pred==1
fig.add_trace(go.Scatter3d(x=X[t,1],y=X[t,2],z=X[t,3], mode='markers', marker=dict(color='blue'), 
                           name='No fraude'), row=1, col=1)
fig.add_trace(go.Scatter3d(x=X[n,1],y=X[n,2],z=X[n,3], mode='markers', marker=dict(color='orange'), 
                           name='Fraude'), row=1, col=1)
t = y==0
n = y==1
fig.add_trace(go.Scatter3d(x=X[t,1],y=X[t,2],z=X[t,3], mode='markers', marker=dict(color='blue'),
                           name = 'No fraude'), row=1, col=2)
fig.add_trace(go.Scatter3d(x=X[n,1],y=X[n,2],z=X[n,3], mode='markers', marker=dict(color='orange'), 
                           name = 'Fraude'), row=1, col=2)
fig.show()

# PCA 

In [ ]:
pca = PCA(n_components=5)
pca.fit(X)
X_pca = pca.transform(X)

In [ ]:
y_pred_pca = SpectralClustering(n_clusters=2,assign_labels='discretize',random_state=0).fit_predict(X_pca)

In [ ]:
cm = confusion_matrix(y, y_pred_pca)

disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=[])
disp.plot()